In [15]:
import pandas as pd
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [16]:
def build_lstm_model(input_shape=(20, 1)):
    model = Sequential()
    model.add(Input(shape=input_shape))       
    model.add(LSTM(64, activation='tanh'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [17]:
#500 temperaturas generadas aleatoriamente
np.random.seed(42)
temperaturas = np.cumsum(np.random.randn(500) * 0.5 + 0.1) + 20  # empieza en 20 y fluctúa

scaler = MinMaxScaler()
temperaturas_norm = scaler.fit_transform(temperaturas.reshape(-1, 1)).flatten()

X, y = [], []
for i in range(len(temperaturas_norm) - 20):
    X.append(temperaturas_norm[i:i+20])
    y.append(temperaturas_norm[i+20])

X = np.array(X)
y_train = np.array(y)
X_train = X.reshape((X.shape[0], X.shape[1], 1))

In [18]:
model = build_lstm_model(input_shape=(20, 1))  
model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.1257 - val_loss: 0.0070
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0142 - val_loss: 0.0347
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0061 - val_loss: 0.0018
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0021 - val_loss: 0.0014
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 7.2221e-04 - val_loss: 6.7687e-04
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.6130e-04 - val_loss: 5.5830e-04
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 4.0735e-04 - val_loss: 0.0012
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.5341e-04 - val_loss: 5.1510e-04
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.3098e-04 - val_loss: 6.4286e-04
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 3.6942e-04 - val_loss: 5.2517e-04
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 3.3491e-04 - val_loss: 5.3866e-04
Epoch 12/50

In [19]:
model.save("modelo_lstm_temperatura.keras")